# NoETL Playbook Execution Report
__Agent 007 mission report:__ _Operation completed. Martini status: shaken, not stirred._
- This notebook provides a report of the execution of a NoETL playbook using the `agent007.py` script.
- It analyzes the DuckDB database generated during the playbook run, with execution flow, step results, loop iterations, task executions, and context data captured throughout the process.
1. Run playbook with `agent007.py`.
2. Set `db_path` below to that file path.
3. Run all paragraphs to inspect tables and data.


In [14]:
import duckdb
import polars as pl
import os
import matplotlib.pyplot as plt
import pandas as pd
import json
import networkx as nx
from datetime import datetime

# Path to DuckDB database
db_path = '../noetl/agent/agent007.duckdb'  # Default path used by agent007.py
# Example: db_path = '/Users/kadyapam/projects/noetl/noetl/noetl/runtime/agent007.duckdb'

if not os.path.exists(db_path) and db_path != ':memory:':
    raise FileNotFoundError(f"DuckDB database not found: {db_path}")

con = duckdb.connect(db_path)

def parse_json_column(df, column_name):
    if column_name in df.columns:
        def safe_json_parse(x):
            if not x:
                return None
            try:
                return json.loads(x)
            except json.JSONDecodeError as e:
                print(f"Warning: Could not parse JSON: {e}")
                return x

        return df.with_columns([
            pl.col(column_name).map_elements(safe_json_parse, return_dtype=pl.Object)
        ])
    return df

# execution ID
latest_execution = con.execute("""
    SELECT execution_id, MAX(timestamp) as latest_time
    FROM event_log
    GROUP BY execution_id
    ORDER BY latest_time DESC
    LIMIT 1
""").fetchone()

if latest_execution:
    execution_id = latest_execution[0]
    print(f"Analyzing execution ID: {execution_id}")
else:
    print("No executions found in the database.")
    execution_id = None


Analyzing execution ID: 41227439-40cf-4361-aa72-e9b24d6a96a5


In [15]:
%load_ext sql
%sql con --alias duckdb

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## List all tables


In [16]:
tables = con.execute("SHOW TABLES").fetchall()
for t in tables:
    print(t[0])


context
event_log
loop_state
step_results
task_results
transitions
workbook
workflow


## Execution Overview

This section provides a high-level overview of the playbook execution.


In [17]:
if execution_id:
    # execution start end times
    execution_times = con.execute("""
        SELECT 
            MIN(timestamp) as start_time,
            MAX(timestamp) as end_time,
            (EXTRACT(EPOCH FROM MAX(timestamp)) - EXTRACT(EPOCH FROM MIN(timestamp))) as duration_seconds
        FROM event_log
        WHERE execution_id = ?
    """, [execution_id]).fetchone()

    # execution statistics
    execution_stats = con.execute("""
        SELECT
            (SELECT COUNT(*) FROM step_results WHERE execution_id = ?) as step_count,
            (SELECT COUNT(*) FROM task_results WHERE execution_id = ?) as task_count,
            (SELECT COUNT(*) FROM loop_state WHERE execution_id = ?) as loop_count,
            (SELECT COUNT(*) FROM event_log WHERE execution_id = ? AND event_type LIKE '%error%') as error_count
    """, [execution_id, execution_id, execution_id, execution_id]).fetchone()

    # playbook name
    playbook_info = con.execute("""
        SELECT node_name, metadata
        FROM event_log
        WHERE execution_id = ? AND event_type = 'execution_start'
        LIMIT 1
    """, [execution_id]).fetchone()

    # execution overview
    print(f"Playbook: {playbook_info[0] if playbook_info else 'Unknown'}")
    print(f"Start Time: {execution_times[0]}")
    print(f"End Time: {execution_times[1]}")
    print(f"Duration: {execution_times[2]:.2f} seconds")
    print(f"Steps Executed: {execution_stats[0]}")
    print(f"Tasks Executed: {execution_stats[1]}")
    print(f"Loops Executed: {execution_stats[2]}")
    print(f"Errors Encountered: {execution_stats[3]}")

    # playbook metadata
    if playbook_info and playbook_info[1]:
        try:
            metadata = json.loads(playbook_info[1])
            print("\nPlaybook Metadata:")
            for key, value in metadata.items():
                print(f"  {key}: {value}")
        except:
            pass
else:
    print("No execution ID available.")


Playbook: weather_iterator_example
Start Time: 2025-06-14 22:02:26.836108
End Time: 2025-06-14 22:02:27.003033
Duration: 0.17 seconds
Steps Executed: 6
Tasks Executed: 11
Loops Executed: 2
Errors Encountered: 0

Playbook Metadata:
  playbook_path: catalog/playbooks/weather_example.yaml


## Step Execution Analysis

Analyze the execution of steps in the playbook.


In [18]:
if execution_id:
    # step execution details
    step_results = pl.from_arrow(con.execute("""
        SELECT 
            step_id, 
            step_name, 
            parent_id, 
            timestamp, 
            status, 
            data, 
            error
        FROM step_results
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    step_results = parse_json_column(step_results, 'data')

    # step execution summary
    step_summary = pl.from_arrow(con.execute("""
        SELECT 
            step_name, 
            COUNT(*) as execution_count,
            COUNT(CASE WHEN status = 'success' THEN 1 END) as success_count,
            COUNT(CASE WHEN status = 'error' THEN 1 END) as error_count
        FROM step_results
        WHERE execution_id = ?
        GROUP BY step_name
        ORDER BY execution_count DESC
    """, [execution_id]).arrow())

    print("Step Execution Summary:")
    display(step_summary)

    # step results
    print("\nStep Results:")
    display(step_results)
else:
    print("No execution ID available.")


Step Execution Summary:


step_name,execution_count,success_count,error_count
str,i64,i64,i64
"""city_loop""",1,1,0
"""aggregate_alerts""",1,1,0
"""district_loop""",1,1,0
"""end_city_loop""",1,1,0
"""end_district_loop""",1,1,0
"""start""",1,1,0



Step Results:


step_id,step_name,parent_id,timestamp,status,data,error
str,str,str,datetime[μs],str,object,str
"""d406fc92-6c4a-43f7-a9db-2dcf2b…","""start""",null,2025-06-14 22:02:26.838742,"""success""",{},null
"""1e911ff3-08a5-4cb0-9b75-616133…","""city_loop""",null,2025-06-14 22:02:26.886133,"""success""","[{'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}]",null
"""46aacd0c-c004-4b7e-a7d1-11a970…","""district_loop""",null,2025-06-14 22:02:26.971321,"""success""","[{'process_district': {'city': 'Berlin', 'district': 'Downtown', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'North', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'East', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}]",null
"""abe6b875-95c1-4a4b-bb0d-f4d774…","""end_district_loop""","""7a54ea75-bb0a-47c7-bcda-f2f26f…",2025-06-14 22:02:26.987217,"""success""",{'district_loop_output': None},null
"""fa5ac692-4278-4994-9ec6-e31fd2…","""end_city_loop""","""77046fc4-ac3b-45f7-bca5-0e425f…",2025-06-14 22:02:26.995681,"""success""",{'alerts': None},null
"""22d08ddc-a4ab-499a-982a-f7cfc2…","""aggregate_alerts""",null,2025-06-14 22:02:27.000586,"""success""",{},null


## Loop Execution Analysis

Analyze the execution of loops in the playbook.


In [19]:
if execution_id:
    # loop execution details
    loop_state = pl.from_arrow(con.execute("""
        SELECT 
            loop_id, 
            loop_name, 
            parent_id, 
            iterator, 
            items, 
            current_index, 
            current_item, 
            results, 
            timestamp, 
            status
        FROM loop_state
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    loop_state = parse_json_column(loop_state, 'items')
    loop_state = parse_json_column(loop_state, 'current_item')
    loop_state = parse_json_column(loop_state, 'results')

    # loop execution summary
    if len(loop_state) > 0:
        print("Loop Execution Summary:")
        display(loop_state)

        # loop iteration events
        loop_events = pl.from_arrow(con.execute("""
            SELECT 
                event_id, 
                node_name, 
                event_type, 
                timestamp, 
                status, 
                duration, 
                output_result as data, 
                metadata
            FROM event_log
            WHERE execution_id = ? AND event_type LIKE '%loop%'
            ORDER BY timestamp
        """, [execution_id]).arrow())

        loop_events = parse_json_column(loop_events, 'data')
        loop_events = parse_json_column(loop_events, 'metadata')

        print("\nLoop Events:")
        display(loop_events)
    else:
        print("No loop executions found.")
else:
    print("No execution ID available.")


Loop Execution Summary:


loop_id,loop_name,parent_id,iterator,items,current_index,current_item,results,timestamp,status
str,str,str,str,object,i32,object,object,datetime[μs],str
"""7a54ea75-bb0a-47c7-bcda-f2f26f…","""district_loop""","""46aacd0c-c004-4b7e-a7d1-11a970…","""district""","[{'name': 'Downtown', 'population': 50000}, {'name': 'North', 'population': 25000}, {'name': 'East', 'population': 30000}, {'name': 'Mordor', 'population': 666}]",4,null,"[{'process_district': {'city': 'Berlin', 'district': 'Downtown', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'North', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'East', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}]",2025-06-14 22:02:26.986806,"""completed"""
"""77046fc4-ac3b-45f7-bca5-0e425f…","""city_loop""","""1e911ff3-08a5-4cb0-9b75-616133…","""city""","[{'name': 'London', 'lat': 51.51, 'lon': -0.13}, {'name': 'Paris', 'lat': 48.85, 'lon': 2.35}, {'name': 'Berlin', 'lat': 52.52, 'lon': 13.41}]",3,null,"[{'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}]",2025-06-14 22:02:26.995191,"""completed"""



Loop Events:


event_id,node_name,event_type,timestamp,status,duration,data,metadata
str,str,str,datetime[μs],str,f64,object,object
"""4a67532c-7af6-4418-a6dd-1537f6…","""city_loop""","""loop_start""",2025-06-14 22:02:26.845878,"""in_progress""",0.0,null,"{'item_count': 3, 'iterator': 'city'}"
"""51562b4b-7d8c-4657-95b9-3c2589…","""city_loop[0]""","""loop_iteration""",2025-06-14 22:02:26.846931,"""in_progress""",0.0,null,"{'index': 0, 'item': {'name': 'London', 'lat': 51.51, 'lon': -0.13}}"
"""be3dd771-68c9-49e5-aab7-806ac0…","""city_loop[0]""","""loop_iteration_complete""",2025-06-14 22:02:26.858187,"""success""",0.0,"{'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}","{'index': 0, 'item': {'name': 'London', 'lat': 51.51, 'lon': -0.13}}"
"""65f68ed8-6faf-41d4-a63e-b9908a…","""city_loop[1]""","""loop_iteration""",2025-06-14 22:02:26.860143,"""in_progress""",0.0,null,"{'index': 1, 'item': {'name': 'Paris', 'lat': 48.85, 'lon': 2.35}}"
"""c94e514e-05ca-4d05-b025-2a247c…","""city_loop[1]""","""loop_iteration_complete""",2025-06-14 22:02:26.871917,"""success""",0.0,"{'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}","{'index': 1, 'item': {'name': 'Paris', 'lat': 48.85, 'lon': 2.35}}"
…,…,…,…,…,…,…,…
"""014938a4-8588-45cc-9446-d21e58…","""district_loop""","""loop_complete""",2025-06-14 22:02:26.972101,"""success""",0.055314,"[{'process_district': {'city': 'Berlin', 'district': 'Downtown', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'North', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'East', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}]","{'item_count': 4, 'processed_count': 4}"
"""9b671cce-dc7c-471d-9db0-89cd62…","""end_district_loop""","""end_loop_start""",2025-06-14 22:02:26.983391,"""in_progress""",0.0,null,{'loop_name': 'district_loop'}
"""913b12fd-a287-45c7-a13f-dfc1b4…","""end_district_loop""","""end_loop_complete""",2025-06-14 22:02:26.987883,"""success""",0.004499,{'district_loop_output': None},{'loop_name': 'district_loop'}


## Task Execution Analysis

Analyze the execution of tasks in the playbook.


In [20]:
if execution_id:
    # task execution details
    task_results = pl.from_arrow(con.execute("""
        SELECT 
            task_id, 
            task_name, 
            task_type, 
            parent_id, 
            timestamp, 
            status, 
            data, 
            error
        FROM task_results
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    task_results = parse_json_column(task_results, 'data')

    # task execution summary
    task_summary = pl.from_arrow(con.execute("""
        SELECT 
            task_name, 
            task_type,
            COUNT(*) as execution_count,
            COUNT(CASE WHEN status = 'success' THEN 1 END) as success_count,
            COUNT(CASE WHEN status = 'error' THEN 1 END) as error_count
        FROM task_results
        WHERE execution_id = ?
        GROUP BY task_name, task_type
        ORDER BY execution_count DESC
    """, [execution_id]).arrow())

    print("Task Execution Summary:")
    display(task_summary)

    # task results
    print("\nTask Results:")
    display(task_results)
else:
    print("No execution ID available.")


Task Execution Summary:


task_name,task_type,execution_count,success_count,error_count
str,str,i64,i64,i64
"""process_district""","""python""",5,5,0
"""get_forecast""","""http""",4,4,0
"""alert_task""","""http""",1,1,0
"""get_city_districts""","""http""",1,1,0



Task Results:


task_id,task_name,task_type,parent_id,timestamp,status,data,error
str,str,str,str,datetime[μs],str,object,str
"""7398a0a4-220a-4dbe-8fff-68391a…","""get_forecast""","""http""","""4d3e0021-539a-4a80-9786-3f9583…",2025-06-14 22:02:26.852828,"""success""","{'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}",null
"""3c34896b-5412-47b8-bcf0-e7294a…","""get_forecast""","""http""","""737e1f95-ca98-4158-a48d-8e882e…",2025-06-14 22:02:26.867596,"""success""","{'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}",null
"""c9054583-cde2-49e2-9ef3-89ba9f…","""get_forecast""","""http""","""81d87b13-543d-4457-82f0-dd3143…",2025-06-14 22:02:26.879839,"""success""","{'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}",null
"""be769540-0ef2-4a72-a001-b360c8…","""get_forecast""","""http""","""35be9f5c-d169-41bf-8ad7-4cfcef…",2025-06-14 22:02:26.891773,"""success""","{'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}",null
"""c86e1483-5f56-4bde-849f-29cd6c…","""alert_task""","""http""","""8d949844-fdf1-4146-8ec2-4d87dd…",2025-06-14 22:02:26.901064,"""success""","{'data': 'mocked_response', 'status': 'success'}",null
…,…,…,…,…,…,…,…
"""65f26e91-ec5e-4737-a216-4c05db…","""process_district""","""python""","""222e774e-2c5c-43e8-8773-c65a53…",2025-06-14 22:02:26.927042,"""success""","{'city': 'Berlin', 'district': 'Downtown', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}",null
"""5c9de3d2-5819-4168-ae2d-333998…","""process_district""","""python""","""9923d2b0-dfe8-429b-bc1a-feb6d1…",2025-06-14 22:02:26.940698,"""success""","{'city': 'Berlin', 'district': 'North', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}",null
"""0c9b94ea-6ad6-4460-b098-31b876…","""process_district""","""python""","""0a3096fd-ab59-444d-9728-06dc04…",2025-06-14 22:02:26.952207,"""success""","{'city': 'Berlin', 'district': 'East', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}",null


## Context Analysis

Analyze the context data during playbook execution.


In [21]:
if execution_id:
    # context data
    context_data = pl.from_arrow(con.execute("""
        SELECT 
            key, 
            value, 
            timestamp
        FROM context
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    context_data = parse_json_column(context_data, 'value')

    if len(context_data) > 0:
        print("Context Data:")
        display(context_data)

        # context state
        final_context = {}
        for _, row in context_data.to_pandas().iterrows():
            final_context[row['key']] = row.get('value')

        print("\nFinal Context State:")
        for key, value in final_context.items():
            print(f"\n{key}:")
            if isinstance(value, (dict, list)):
                print(json.dumps(value, indent=2))
            else:
                print(value)
    else:
        print("No context data found.")
else:
    print("No execution ID found.")


Context Data:


key,value,timestamp
str,object,datetime[μs]
"""workload""","{'jobId': '{{ job.uuid }}', 'state': 'ready', 'cities': [{'name': 'London', 'lat': 51.51, 'lon': -0.13}, {'name': 'Paris', 'lat': 48.85, 'lon': 2.35}, {'name': 'Berlin', 'lat': 52.52, 'lon': 13.41}], 'base_url': 'https://api.open-meteo.com/v1', 'temperature_threshold': 25}",2025-06-14 22:02:26.834306
"""execution_start""",2025-06-14T22:02:26.835735,2025-06-14 22:02:26.835744
"""base_url""",https://api.open-meteo.com/v1,2025-06-14 22:02:26.875025
"""temperature_threshold""",25,2025-06-14 22:02:26.875614
"""city_loop_results""","[{'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}]",2025-06-14 22:02:26.885678
…,…,…
"""district_loop_results""","[{'process_district': {'city': 'Berlin', 'district': 'Downtown', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'North', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'East', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}]",2025-06-14 22:02:26.970804
"""process_district""","{'result': {'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}, 'status': 'success'}",2025-06-14 22:02:26.979711
"""result""","{'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}",2025-06-14 22:02:26.980079



Final Context State:

workload:
{
  "jobId": "{{ job.uuid }}",
  "state": "ready",
  "cities": [
    {
      "name": "London",
      "lat": 51.51,
      "lon": -0.13
    },
    {
      "name": "Paris",
      "lat": 48.85,
      "lon": 2.35
    },
    {
      "name": "Berlin",
      "lat": 52.52,
      "lon": 13.41
    }
  ],
  "base_url": "https://api.open-meteo.com/v1",
  "temperature_threshold": 25
}

execution_start:
2025-06-14T22:02:26.835735

base_url:
https://api.open-meteo.com/v1

temperature_threshold:
25

city_loop_results:
[
  {
    "fetch_and_evaluate": {
      "data": {
        "hourly": {
          "temperature_2m": [
            20,
            22,
            25,
            28,
            30,
            26,
            24
          ],
          "precipitation_probability": [
            0,
            10,
            20,
            30,
            20,
            10,
            0
          ],
          "windspeed_10m": [
            10,
            12,
            

## Preview `event_log` table


In [22]:
table_name = 'event_log'
df = pl.from_arrow(con.execute(f"SELECT * FROM {table_name} LIMIT 20").arrow())
df


execution_id,event_id,parent_event_id,timestamp,event_type,node_id,node_name,node_type,status,duration,input_context,output_result,metadata
str,str,str,datetime[μs],str,str,str,str,str,f64,str,str,str
"""7586571c-d367-4176-880c-5d3c9e…","""cbc63052-c6b4-4e2c-8403-6e0c98…",null,2025-06-14 21:53:56.355225,"""execution_start""","""7586571c-d367-4176-880c-5d3c9e…","""weather_iterator_example""","""playbook""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""playbook_path"": ""catalog/pla…"
"""7586571c-d367-4176-880c-5d3c9e…","""4c959502-d2bd-4597-a3ef-8dd5dd…",null,2025-06-14 21:53:56.356253,"""step_start""","""ec23ce30-451f-425e-b349-36e002…","""start""","""step""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""step_type"": ""standard""}"""
"""7586571c-d367-4176-880c-5d3c9e…","""77862cae-d698-4266-92c9-0b8f60…","""4c959502-d2bd-4597-a3ef-8dd5dd…",2025-06-14 21:53:56.357608,"""step_complete""","""ec23ce30-451f-425e-b349-36e002…","""start""","""step""","""success""",0.001457,"""{""jobId"": ""{{ job.uuid }}"", ""s…","""{}""","""{""step_type"": ""standard""}"""
"""7586571c-d367-4176-880c-5d3c9e…","""c9036945-10b2-4eba-8c5a-72be64…","""cbc63052-c6b4-4e2c-8403-6e0c98…",2025-06-14 21:53:56.360387,"""step_transition""","""7586571c-d367-4176-880c-5d3c9e…","""transition_to_city_loop""","""transition""","""success""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""from_step"": ""start"", ""to_ste…"
"""7586571c-d367-4176-880c-5d3c9e…","""50bcaa34-78d7-4577-bf3e-0b4d28…",null,2025-06-14 21:53:56.361025,"""step_start""","""29292739-01e1-417f-8f81-7a7d3e…","""city_loop""","""step""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""step_type"": ""standard""}"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""7586571c-d367-4176-880c-5d3c9e…","""3ee860dd-404e-451a-9e9e-96baf1…",null,2025-06-14 21:53:56.381947,"""step_start""","""c0ac7068-10fa-4077-b5c9-60a689…","""fetch_and_evaluate""","""step""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""step_type"": ""standard""}"""
"""7586571c-d367-4176-880c-5d3c9e…","""7f3e9be9-36b2-4396-9065-dff44a…","""3ee860dd-404e-451a-9e9e-96baf1…",2025-06-14 21:53:56.382723,"""task_execute""","""852b9219-bc6d-44bf-ac96-10cfe0…","""get_forecast""","""task.http""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""task_type"": ""http""}"""
"""7586571c-d367-4176-880c-5d3c9e…","""61fea685-5903-454f-a8ad-f1c181…","""7f3e9be9-36b2-4396-9065-dff44a…",2025-06-14 21:53:56.383952,"""task_start""","""e958fa21-6a61-4455-92a7-b4b71c…","""get_forecast""","""http""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""method"": ""GET"", ""endpoint"": …"


## Show all steps resluts


In [23]:
pl.from_arrow(con.execute("SELECT * FROM step_results ORDER BY timestamp DESC LIMIT 10").arrow())


execution_id,step_id,step_name,parent_id,timestamp,status,data,error
str,str,str,str,datetime[μs],str,str,str
"""41227439-40cf-4361-aa72-e9b24d…","""22d08ddc-a4ab-499a-982a-f7cfc2…","""aggregate_alerts""",null,2025-06-14 22:02:27.000586,"""success""","""{}""",null
"""41227439-40cf-4361-aa72-e9b24d…","""fa5ac692-4278-4994-9ec6-e31fd2…","""end_city_loop""","""77046fc4-ac3b-45f7-bca5-0e425f…",2025-06-14 22:02:26.995681,"""success""","""{""alerts"": null}""",null
"""41227439-40cf-4361-aa72-e9b24d…","""abe6b875-95c1-4a4b-bb0d-f4d774…","""end_district_loop""","""7a54ea75-bb0a-47c7-bcda-f2f26f…",2025-06-14 22:02:26.987217,"""success""","""{""district_loop_output"": null}""",null
"""41227439-40cf-4361-aa72-e9b24d…","""46aacd0c-c004-4b7e-a7d1-11a970…","""district_loop""",null,2025-06-14 22:02:26.971321,"""success""","""[{""process_district"": {""city"":…",null
"""41227439-40cf-4361-aa72-e9b24d…","""1e911ff3-08a5-4cb0-9b75-616133…","""city_loop""",null,2025-06-14 22:02:26.886133,"""success""","""[{""fetch_and_evaluate"": {""data…",null
"""41227439-40cf-4361-aa72-e9b24d…","""d406fc92-6c4a-43f7-a9db-2dcf2b…","""start""",null,2025-06-14 22:02:26.838742,"""success""","""{}""",null
"""7586571c-d367-4176-880c-5d3c9e…","""cf22a93f-8eba-402b-8a5e-448a57…","""aggregate_alerts""",null,2025-06-14 21:53:56.489057,"""success""","""{}""",null
"""7586571c-d367-4176-880c-5d3c9e…","""199c5008-9354-4418-b3e9-8b646c…","""end_city_loop""","""8286313c-74ad-45cc-bdcf-8d2097…",2025-06-14 21:53:56.484202,"""success""","""{""alerts"": null}""",null
"""7586571c-d367-4176-880c-5d3c9e…","""69ebb59a-8a99-4c1d-92c1-4e7fb7…","""end_district_loop""","""218c3a5a-39d7-40a2-b318-6c85d6…",2025-06-14 21:53:56.474956,"""success""","""{""district_loop_output"": null}""",null


## Show all task results

In [24]:
con.sql("SELECT * FROM task_results ORDER BY timestamp DESC LIMIT 10").show()

┌──────────────────────────────────────┬──────────────────────────────────────┬────────────────────┬───────────┬──────────────────────────────────────┬────────────────────────────┬─────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────┐
│             execution_id             │               task_id                │     task_name      │ task_type │              parent_id               │         timestamp          │ status  │                                                                                                     data                                                                                                      │  error  │
│               varchar                │               varchar                │      varchar       │  varchar  │               varchar                │         timestamp          │ v

In [25]:
%sql SELECT * FROM task_results ORDER BY timestamp DESC LIMIT 10

Running query in 'duckdb'

execution_id,task_id,task_name,task_type,parent_id,timestamp,status,data,error
41227439-40cf-4361-aa72-e9b24d6a96a5,b877935b-7273-4e5e-8966-55e66505b435,process_district,python,5a6a414a-5847-4cb7-a40b-3a321d4ee095,2025-06-14 22:02:26.977749,success,"{""city"": ""Berlin"", ""district"": ""Mordor"", ""processed"": true, ""timestamp"": ""2024-01-01T12:00:00Z""}",None
41227439-40cf-4361-aa72-e9b24d6a96a5,3f9e689e-96ce-406d-9b3b-fd51113390bd,process_district,python,c443080f-d2d9-4fb0-b7ed-4762860551cc,2025-06-14 22:02:26.964501,success,"{""city"": ""Berlin"", ""district"": ""Mordor"", ""processed"": true, ""timestamp"": ""2024-01-01T12:00:00Z""}",None
41227439-40cf-4361-aa72-e9b24d6a96a5,0c9b94ea-6ad6-4460-b098-31b876dccdaa,process_district,python,0a3096fd-ab59-444d-9728-06dc042b456a,2025-06-14 22:02:26.952207,success,"{""city"": ""Berlin"", ""district"": ""East"", ""processed"": true, ""timestamp"": ""2024-01-01T12:00:00Z""}",None
41227439-40cf-4361-aa72-e9b24d6a96a5,5c9de3d2-5819-4168-ae2d-333998c4b8f1,process_district,python,9923d2b0-dfe8-429b-bc1a-feb6d19e4136,2025-06-14 22:02:26.940698,success,"{""city"": ""Berlin"", ""district"": ""North"", ""processed"": true, ""timestamp"": ""2024-01-01T12:00:00Z""}",None
41227439-40cf-4361-aa72-e9b24d6a96a5,65f26e91-ec5e-4737-a216-4c05db45562d,process_district,python,222e774e-2c5c-43e8-8773-c65a53065e4f,2025-06-14 22:02:26.927042,success,"{""city"": ""Berlin"", ""district"": ""Downtown"", ""processed"": true, ""timestamp"": ""2024-01-01T12:00:00Z""}",None
41227439-40cf-4361-aa72-e9b24d6a96a5,b6856eac-d040-4355-b3d0-146390007976,get_city_districts,http,8f636190-11ba-4d6a-a4c0-97085cf18e42,2025-06-14 22:02:26.909698,success,"{""data"": [{""name"": ""Downtown"", ""population"": 50000}, {""name"": ""North"", ""population"": 25000}, {""name"": ""East"", ""population"": 30000}, {""name"": ""Mordor"", ""population"": 666}]}",None
41227439-40cf-4361-aa72-e9b24d6a96a5,c86e1483-5f56-4bde-849f-29cd6c414242,alert_task,http,8d949844-fdf1-4146-8ec2-4d87dd9a28a8,2025-06-14 22:02:26.901064,success,"{""data"": ""mocked_response"", ""status"": ""success""}",None
41227439-40cf-4361-aa72-e9b24d6a96a5,be769540-0ef2-4a72-a001-b360c8e7e389,get_forecast,http,35be9f5c-d169-41bf-8ad7-4cfcef02e394,2025-06-14 22:02:26.891773,success,"{""data"": {""hourly"": {""temperature_2m"": [20, 22, 25, 28, 30, 26, 24], ""precipitation_probability"": [0, 10, 20, 30, 20, 10, 0], ""windspeed_10m"": [10, 12, 15, 18, 22, 19, 14]}}, ""alert"": true, ""max_temp"": 30}",None
41227439-40cf-4361-aa72-e9b24d6a96a5,c9054583-cde2-49e2-9ef3-89ba9f88db34,get_forecast,http,81d87b13-543d-4457-82f0-dd3143ff66ed,2025-06-14 22:02:26.879839,success,"{""data"": {""hourly"": {""temperature_2m"": [20, 22, 25, 28, 30, 26, 24], ""precipitation_probability"": [0, 10, 20, 30, 20, 10, 0], ""windspeed_10m"": [10, 12, 15, 18, 22, 19, 14]}}, ""alert"": true, ""max_temp"": 30}",None
41227439-40cf-4361-aa72-e9b24d6a96a5,3c34896b-5412-47b8-bcf0-e7294aff6002,get_forecast,http,737e1f95-ca98-4158-a48d-8e882e300f94,2025-06-14 22:02:26.867596,success,"{""data"": {""hourly"": {""temperature_2m"": [20, 22, 25, 28, 30, 26, 24], ""precipitation_probability"": [0, 10, 20, 30, 20, 10, 0], ""windspeed_10m"": [10, 12, 15, 18, 22, 19, 14]}}, ""alert"": true, ""max_temp"": 30}",None
